In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
%run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

data = Data('s', save_generated_data=False)
#data.generate_data('shuffled_syllable_reversed_train',
#                   'shuffled_syllable_reversed_test',
#                   'shuffled_syllable_reversed_validate')
data.generate_data('TEST_train',
                   'TEST_test',
                   'TEST_validate')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [4]:
%run ../../../prepare_data.py
# gen1 = generate_fake_epoch_syllables(X_train[:20], X_other_features_train[:20], y_train[:20], 16, 5168)
# gen2 = generate_fake_epoch_syllables(data.x_train[:20], data.x_other_features_train[:20], data.y_train[:20], 16, 5168)
gen2 = data.generator('train', 16)
# test1 = next(gen1)
test2 = next(gen2)

In [7]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
pos = 0
# print(np.array(test1[0][0]).shape)
# for el in test1[0][0][pos]:
#     print(el)
#     print(syllable_dictionary[el.tolist().index(1)])
# print('-----------------------------------------------')
print(np.array(test2[0][0]).shape)
for el in test2[0][0][pos]:
    print(el)
    print(syllable_dictionary[el.tolist().index(1)])
    
print(data.decode_y(data.y_train[pos]))
print(data.y_train[pos])

(16, 10, 5168)
[0 0 0 ..., 0 0 0]
ša
[0 0 0 ..., 0 0 0]
lnej
[0 0 1 ..., 0 0 0]
a
[0 0 0 ..., 0 0 0]
ktu
[0 0 0 ..., 0 0 0]
ja
[0 0 0 ..., 0 0 0]
na
[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[2, 5]
[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0.]


In [4]:
data.x_train = np.concatenate((data.x_train, data.x_test), axis=0)
data.x_other_features_train = np.concatenate((data.x_other_features_train, data.x_other_features_test), axis=0)
data.y_train = np.concatenate((data.y_train, data.y_test), axis=0)

In [5]:
num_examples = len(data.x_train) # training set size

nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20



conv_input_shape=(10, 5168)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
# x_conv = Conv1D(44, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)





model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [6]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch,
                              verbose=2
                            )

Epoch 1/400
187s - loss: 0.1993 - actual_accuracy: 0.4477
Epoch 2/400
229s - loss: 0.1191 - actual_accuracy: 0.6836
Epoch 3/400
251s - loss: 0.0913 - actual_accuracy: 0.7583
Epoch 4/400
269s - loss: 0.0745 - actual_accuracy: 0.8102
Epoch 5/400
275s - loss: 0.0629 - actual_accuracy: 0.8439
Epoch 6/400
283s - loss: 0.0537 - actual_accuracy: 0.8676
Epoch 7/400
300s - loss: 0.0472 - actual_accuracy: 0.8864
Epoch 8/400
305s - loss: 0.0421 - actual_accuracy: 0.8981
Epoch 9/400
339s - loss: 0.0395 - actual_accuracy: 0.9054
Epoch 10/400
319s - loss: 0.0338 - actual_accuracy: 0.9193
Epoch 11/400
322s - loss: 0.0319 - actual_accuracy: 0.9222
Epoch 12/400
329s - loss: 0.0304 - actual_accuracy: 0.9275
Epoch 13/400
337s - loss: 0.0283 - actual_accuracy: 0.9348
Epoch 14/400
340s - loss: 0.0263 - actual_accuracy: 0.9378
Epoch 15/400
348s - loss: 0.0243 - actual_accuracy: 0.9442
Epoch 16/400
351s - loss: 0.0228 - actual_accuracy: 0.9479
Epoch 17/400
359s - loss: 0.0214 - actual_accuracy: 0.9509
Epoch 

525s - loss: 0.0018 - actual_accuracy: 0.9967
Epoch 141/400
511s - loss: 0.0033 - actual_accuracy: 0.9933
Epoch 142/400
500s - loss: 0.0031 - actual_accuracy: 0.9939
Epoch 143/400
501s - loss: 0.0028 - actual_accuracy: 0.9942
Epoch 144/400
503s - loss: 0.0025 - actual_accuracy: 0.9948
Epoch 145/400
498s - loss: 0.0027 - actual_accuracy: 0.9945
Epoch 146/400
509s - loss: 0.0026 - actual_accuracy: 0.9948
Epoch 147/400
506s - loss: 0.0028 - actual_accuracy: 0.9943
Epoch 148/400
505s - loss: 0.0024 - actual_accuracy: 0.9951
Epoch 149/400
505s - loss: 0.0027 - actual_accuracy: 0.9949
Epoch 150/400
500s - loss: 0.0026 - actual_accuracy: 0.9950
Epoch 151/400
503s - loss: 0.0027 - actual_accuracy: 0.9944
Epoch 152/400
506s - loss: 0.0025 - actual_accuracy: 0.9949
Epoch 153/400
502s - loss: 0.0023 - actual_accuracy: 0.9952
Epoch 154/400
499s - loss: 0.0026 - actual_accuracy: 0.9947
Epoch 155/400
506s - loss: 0.0022 - actual_accuracy: 0.9953
Epoch 156/400
506s - loss: 0.0028 - actual_accuracy: 0

540s - loss: 0.0012 - actual_accuracy: 0.9974
Epoch 278/400
538s - loss: 0.0020 - actual_accuracy: 0.9965
Epoch 279/400
556s - loss: 0.0013 - actual_accuracy: 0.9978
Epoch 280/400
535s - loss: 0.0010 - actual_accuracy: 0.9981
Epoch 281/400
531s - loss: 0.0019 - actual_accuracy: 0.9962
Epoch 282/400
534s - loss: 0.0016 - actual_accuracy: 0.9967
Epoch 283/400
536s - loss: 0.0013 - actual_accuracy: 0.9975
Epoch 284/400
527s - loss: 0.0012 - actual_accuracy: 0.9971
Epoch 285/400
539s - loss: 0.0014 - actual_accuracy: 0.9977
Epoch 286/400
530s - loss: 0.0013 - actual_accuracy: 0.9974
Epoch 287/400
570s - loss: 0.0013 - actual_accuracy: 0.9972
Epoch 288/400
533s - loss: 0.0014 - actual_accuracy: 0.9972
Epoch 289/400
531s - loss: 0.0015 - actual_accuracy: 0.9965
Epoch 290/400
534s - loss: 0.0015 - actual_accuracy: 0.9966
Epoch 291/400
535s - loss: 0.0014 - actual_accuracy: 0.9973
Epoch 292/400
526s - loss: 0.0013 - actual_accuracy: 0.9978
Epoch 293/400
537s - loss: 0.0012 - actual_accuracy: 0

In [8]:
name = '20_test_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [6]:
# dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
# train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
# feature_dictionary = create_feature_dictionary()
# syllable_dictionary = create_syllables_dictionary(content, vowels)


content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)